## Draw a Cube with PySide2 OpenGL

Welcome to PySide2 OpenGL cube tutorial. 

Why draw a cube ? So far we had seen only 2d rendering. 
A cube introduces us to 3d rendering in OpenGL. 
With respect to 2d rendering it introduces several additional configurations that need to be taken into account which are tied to the very nature of 3d objects. These are:

- 3d objects are contained in a 3d world.
- 3d objects are observed from somewhere inside the world.
- Observation process transforms the 3d object to a 2d object.

Now these three facts obliges to use three additional objects in shaders to render a 3d object:

- **model** matrix 4x4: determines the position of the object in a 3d world
- **view** matrix 4x4: transforms the coordinates of objects in a 3d world with respect to the camera/viewer position
- **projection** matrix 4x4: determines how object(s) would appear with respect to the position and the direction of the viewer.

It should be more or less clear that 3d rendering needs a little more consideration and effort. 

Let's see the final form of the application.

In [2]:
import subprocess

subprocess.run(["python", "app.py"])

CompletedProcess(args=['python', 'app.py'], returncode=0)

All right! As usual, I skip the window code and concentrate on the GL widget instead.

Let's see the constructor of our widget.

In [ ]:
class CubeGL(QOpenGLWidget):
    "Cube gl widget"

    def __init__(self, parent=None):
        QOpenGLWidget.__init__(self, parent)

        ############# Diff ###############
        # We represent the viewer as a camera
        # The code can be a little strange
        # to those who are not used to graphics
        # programming. In our helper code,
        # we had provided a pure python 
        # implementation and a Qt version
        # which uses qt objects.
        
        # The values we set here are important
        # for setting up the view matrix
        # camera
        self.camera = QtCamera()
        self.camera.position = QVector3D(0.0, 0.0, 3.0)
        self.camera.front = QVector3D(0.0, 0.0, -1.0)
        self.camera.up = QVector3D(0.0, 1.0, 0.0)

        # shaders etc
        tutoTutoDir = os.path.dirname(__file__)
        tutoPardir = os.path.join(tutoTutoDir, os.pardir)
        tutoPardir = os.path.realpath(tutoPardir)
        mediaDir = os.path.join(tutoPardir, "media")
        shaderDir = os.path.join(mediaDir, "shaders")

        availableShaders = ["cube"]
        self.shaders = {
            name: {
                "fragment": os.path.join(shaderDir, name + ".frag"),
                "vertex": os.path.join(shaderDir, name + ".vert")
            } for name in availableShaders
        }
        self.core = "--coreprofile" in QCoreApplication.arguments()
        imdir = os.path.join(mediaDir, "images")
        imFName = "im"
        imageFile1 = os.path.join(imdir, imFName + "0.png")
        self.image1 = QImage(imageFile1).mirrored()
        
        ################ Diff #################
        # We are going to use 2 textures.
        # we shall see usage differences
        # as we go along the code.
        imageFile2 = os.path.join(imdir, imFName + "1.png")
        self.image2 = QImage(imageFile2).mirrored()

        # opengl data related
        self.context = QOpenGLContext()
        self.vao = QOpenGLVertexArrayObject()
        self.vbo = QOpenGLBuffer(QOpenGLBuffer.VertexBuffer)
        self.program = QOpenGLShaderProgram()
        self.texture1 = None
        self.texture2 = None
        self.texUnit1 = 0
        self.texUnit2 = 1

        ############## Diff ##############
        # cube is made up of 6 sides each side 
        # is a square which is made up of 2
        # triangles and for each triangle we
        # specify 3 corners
        self.cubeVertices = np.array([
            # pos vec3 || texcoord vec2
            -0.5, -0.5, -0.5, 0.0, 0.0,
            0.5, -0.5, -0.5, 1.0, 0.0,
            0.5,  0.5, -0.5,  1.0, 1.0,
            0.5,  0.5, -0.5,  1.0, 1.0,
            -0.5,  0.5, -0.5,  0.0, 1.0,
            -0.5, -0.5, -0.5,  0.0, 0.0,

            -0.5, -0.5,  0.5,  0.0, 0.0,
            0.5, -0.5,  0.5,  1.0, 0.0,
            0.5,  0.5,  0.5,  1.0, 1.0,
            0.5,  0.5,  0.5,  1.0, 1.0,
            -0.5,  0.5,  0.5,  0.0, 1.0,
            -0.5, -0.5,  0.5,  0.0, 0.0,

            -0.5,  0.5,  0.5,  1.0, 0.0,
            -0.5,  0.5, -0.5,  1.0, 1.0,
            -0.5, -0.5, -0.5,  0.0, 1.0,
            -0.5, -0.5, -0.5,  0.0, 1.0,
            -0.5, -0.5,  0.5,  0.0, 0.0,
            -0.5,  0.5,  0.5,  1.0, 0.0,

            0.5,  0.5,  0.5,  1.0, 0.0,
            0.5,  0.5, -0.5,  1.0, 1.0,
            0.5, -0.5, -0.5,  0.0, 1.0,
            0.5, -0.5, -0.5,  0.0, 1.0,
            0.5, -0.5,  0.5,  0.0, 0.0,
            0.5,  0.5,  0.5,  1.0, 0.0,

            -0.5, -0.5, -0.5,  0.0, 1.0,
            0.5, -0.5, -0.5,  1.0, 1.0,
            0.5, -0.5,  0.5,  1.0, 0.0,
            0.5, -0.5,  0.5,  1.0, 0.0,
            -0.5, -0.5,  0.5,  0.0, 0.0,
            -0.5, -0.5, -0.5,  0.0, 1.0,

            -0.5,  0.5, -0.5,  0.0, 1.0,
            0.5,  0.5, -0.5,  1.0, 1.0,
            0.5,  0.5,  0.5,  1.0, 0.0,
            0.5,  0.5,  0.5,  1.0, 0.0,
            -0.5,  0.5,  0.5,  0.0, 0.0,
            -0.5,  0.5, -0.5,  0.0, 1.0
        ], dtype=ctypes.c_float
        )
        ############ Diff ##############
        # As we can see there are 10 cubes.
        # the cubes shape is described by the
        # cubeVertices.
        # We are going to render in the world.
        # These are their positions.
        self.cubeCoords = [
            QVector3D(0.0,  0.0,  0.0),
            QVector3D(2.0,  5.0, -15.0),
            QVector3D(-1.5, -2.2, -2.5),
            QVector3D(-3.8, -2.0, -12.3),
            QVector3D(2.4, -0.4, -3.5),
            QVector3D(-1.7,  3.0, -7.5),
            QVector3D(1.3, -2.0, -2.5),
            QVector3D(1.5,  2.0, -2.5),
            QVector3D(1.5,  0.2, -1.5),
            QVector3D(-1.3,  1.0, -1.5)
        ]

Please inspect the code of `camera.py` when you have the time.
The pure implementation should give you a rough idea about how everything works.
Qt implementation simply facilitates the operations defined in the pure one by using qt objects.

You should note that it is not very feasible to draw anything that is not geometric by specifying the vertices by hand. 
If you are looking to render real objects you should think about using a 3d modelling software like blender for example.

Now let's see the initialization code.

In [ ]:
    def initializeGL(self):
        print('gl initial')
        print(self.getGlInfo())

        # create context and make it current
        self.context.create()
        self.context.aboutToBeDestroyed.connect(
            self.cleanUpGl)

        # initialize functions
        funcs = self.context.functions()
        funcs.initializeOpenGLFunctions()
        funcs.glClearColor(0.0, 0.4, 0.4, 0)
        ################## Diff ####################
        # This readies opengl to render 3d graphics
        funcs.glEnable(pygl.GL_DEPTH_TEST)
        #
        funcs.glEnable(pygl.GL_TEXTURE_2D)

        # cube shader
        self.program = QOpenGLShaderProgram(
            self.context
        )
        vshader = self.loadVertexShader("cube")
        fshader = self.loadFragmentShader("cube")
        self.program.addShader(vshader)  # adding vertex shader
        self.program.addShader(fshader)  # adding fragment shader
        self.program.bindAttributeLocation(
            "aPos", 0)
        self.program.bindAttributeLocation(
            "aTexCoord", 1)

        isLinked = self.program.link()
        print("cube shader program is linked: ",
              isLinked)
        # bind the program
        self.program.bind()

        ############### Diff ###############
        # We define the projection matrix here
        # for its function see our introduction
        # set projection matrix. Please do note
        # that we are using a perspective 
        # projection yet this is not the only
        # projection type that is available
        projectionMatrix = QMatrix4x4()
        projectionMatrix.perspective(
            self.camera.zoom,
            self.width() / self.height(),
            0.2, 100.0)
        
        # we set its value just like any other
        # uniform
        self.program.setUniformValue('projection',
                                     projectionMatrix)

        ############## Diff ##################
        # As stated above, camera, being a fps
        # style camera, gives us the matrix
        # that help us to transform all the other
        # coordinates with respect to the viewer.
        # Meaning that the viewer is considered 
        # as the center and all other coordinates
        # are redefined with respect to that coordinate
        # Notice that this does not mean we have to redefine
        # each object individually we just need to 
        # have the matrix that would give us 
        # the stated coordinate when we apply
        # the transformation.
        # set view/camera matrix
        viewMatrix = self.camera.getViewMatrix()
        self.program.setUniformValue('view',
                                     viewMatrix)
        
        ################# Diff ####################
        # The numbers set here are units for the samplers
        # they are going to be important when we are creating
        # the textures
        self.program.setUniformValue('myTexture1', self.texUnit1)
        self.program.setUniformValue('myTexture2', self.texUnit2)
        #
        # deal with vaos and vbo
        # vbo
        isVbo = self.vbo.create()
        isVboBound = self.vbo.bind()

        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate space on vbo buffer
        self.vbo.allocate(
            self.cubeVertices.tobytes(),
            floatSize * self.cubeVertices.size)
        
        # contary to texture tutorial
        # we are going to reuse the old method 
        # for creating the vertex array objects
        self.vao.create()
        
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao)
        funcs.glEnableVertexAttribArray(0)  # viewport
        funcs.glVertexAttribPointer(0,
                                    3,
                                    int(pygl.GL_FLOAT),
                                    int(pygl.GL_FALSE),
                                    5 * floatSize,
                                    VoidPtr(0)
                                    )
        funcs.glEnableVertexAttribArray(1)
        funcs.glVertexAttribPointer(1,
                                    2,
                                    int(pygl.GL_FLOAT),
                                    int(pygl.GL_FALSE),
                                    5 * floatSize,
                                    VoidPtr(3 * floatSize)
                                    )
        # deal with textures
        # first texture
        self.texture1 = QOpenGLTexture(
            QOpenGLTexture.Target2D)
        self.texture1.create()
        ################# Diff ##################
        # We bind the texture to a specific
        # unit. This is necessary for the
        # sampler. 
        self.texture1.bind(self.texUnit1)
        self.texture1.setData(self.image1)
        self.texture1.setMinMagFilters(
            QOpenGLTexture.Nearest,
            QOpenGLTexture.Nearest)
        self.texture1.setWrapMode(
            QOpenGLTexture.DirectionS,
            QOpenGLTexture.Repeat)
        self.texture1.setWrapMode(
            QOpenGLTexture.DirectionT,
            QOpenGLTexture.Repeat)

        # second texture
        self.texture2 = QOpenGLTexture(
            QOpenGLTexture.Target2D)
        self.texture2.create()
        self.texture2.bind(self.texUnit2)
        self.texture2.setData(self.image2)
        self.texture2.setMinMagFilters(
            QOpenGLTexture.Linear,
            QOpenGLTexture.Linear)
        self.texture2.setWrapMode(
            QOpenGLTexture.DirectionS,
            QOpenGLTexture.Repeat)
        self.texture2.setWrapMode(
            QOpenGLTexture.DirectionT,
            QOpenGLTexture.Repeat)

        self.vbo.release()
        vaoBinder = None
        print("gl initialized")

It might be a little daunting to take all that at once.
But most of it should be familiar by now if you have followed previous tutorials.

Now let's see the drawing loop.

In [ ]:
    def paintGL(self):
        "drawing loop"
        funcs = self.context.functions()

        # clean up what was drawn
        ############### Diff ##############
        # Notice we clear the depth buffer 
        # as well
        funcs.glClear(
            pygl.GL_COLOR_BUFFER_BIT | pygl.GL_DEPTH_BUFFER_BIT
        )
        self.vao.bind()
        self.vbo.bind()

        # actual drawing
        self.program.bind()
        
        ################## Diff ###################
        # This is the first time we are drawing
        # several objects. So we are going to call 
        # several glDrawArrays several times.
        # Notice also that we are not changing the
        # shape of the cube we are simply changing
        # its rotation. The translate function
        # simply multiplies the model matrix
        # with a translation 4x4 matrix constructed
        # from the position 3d vector. Translation
        # matrix is essentially same except for
        # its last column whose first three rows
        # are made up of vectors components
        rotvec = QVector3D(0.7, 0.2, 0.5)
        # bind textures
        for i, pos in enumerate(self.cubeCoords):
            #
            cubeModel = QMatrix4x4()
            cubeModel.translate(pos)
            angle = 30 * i
            cubeModel.rotate(angle, rotvec)
            self.program.setUniformValue("model",
                                         cubeModel)
            self.texture1.bind(self.texUnit1)
            self.texture2.bind(self.texUnit2)
            funcs.glDrawArrays(
                pygl.GL_TRIANGLES,
                0,
                self.cubeVertices.size
            )
        self.vbo.release()
        self.program.release()
        self.texture1.release()
        self.texture2.release()

That's it. 

We have covered a lot of grounds in this tutorial. 
It is normal if you are a bit lost.

If you are not sure how all transformations work,
checkout the section from <a href="https://learnopengl.com/Getting-started/Coordinate-Systems">LearnOpenGL</a>. 

If you want to know more about how a camera works,
checkout from the <a href="https://learnopengl.com/Getting-started/Camera">same</a>.